In [ ]:
!pip install transformers

In [ ]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
datasets = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
device = "cuda:0"
pipe = pipeline("question-answering", model="eragon0206/finetunedforQnA", device = device)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
datasets["validation"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [ ]:
preds = []
acts = []

In [ ]:
!pip install torchinfo

In [ ]:
# Load model directly
'''
Printing the model parameters, the answers to the questions can be found in the report
'''
from torchinfo import summary
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("eragon0206/finetunedforQnA")
print("<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>")
print("<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>")
print(summary(model))


<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>
Layer (type:depth-idx)                                  Param #
BertForQuestionAnswering                                --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
├─Linear: 1-2                                           1,538
Total params: 108,893,186
Trainable params: 108,893,186
Non-trainable params: 0


In [ ]:
for i in range(1500):
  if(len(datasets["validation"]["answers"][i]['text']) == 0):continue
  preds.append(pipe(context = datasets["validation"]["context"][i], question = datasets["validation"]["question"][i])['answer'])
  acts.append(datasets["validation"]["answers"][i]['text'][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(acts, preds, average='micro')

print(f"F1 Score: {f1}")


F1 Score: 0.001326259946949602


In [ ]:
!pip install nltk


In [ ]:
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
total = 0
for i in range(len(acts)):

  act = word_tokenize(acts[i])
  pred = word_tokenize(preds[i])
  total += meteor_score.meteor_score([act], pred)

avg_meteor_score = total/len(acts)
print(avg_meteor_score)

0.05448370782090545


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = [sentence_bleu(ref, pred.split()) for ref, pred in zip(acts, preds)]

average_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"BLEU Scores: {bleu_scores}")
print(f"Average BLEU Score: {average_bleu}")


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU Scores: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0003688322288243e-231, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0244914152188952e-231, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.2183324802375697e-231, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.1200407237786664e-231, 0, 0, 1.1640469867513693e-231, 0, 0, 0, 0, 0, 0, 0, 0, 1.2882297539194154e-231, 0, 0, 0, 0, 9.594503055152632e-232, 0, 9.788429383461836e-232, 0, 9.788429383461836e-232, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9.788429383461836e-232, 0, 9.788429383461836e-232, 0, 0, 0, 0, 0, 0, 1.384292958842266e-231, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.2183324802375697e-231, 1.5319719891192393e-231, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9.594503055152632e-232, 0, 0, 0, 0, 0, 1.0003688322288243e-231,

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=90eb06d252aec12a4f4c59fd18b9f5109cbeb93299838d149a1fe058898cf83b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
score_1 = 0
score_L = 0
for i in range(len(acts)):

  ans = scorer.score(acts[i], preds[i])
  score_1 += ans['rouge1'][2]
  score_L += ans['rougeL'][2]




In [ ]:
score_1 /= score_1/len(acts)
score_L /= score_L/len(acts)

In [ ]:
 #fmeasure scores
print(score_1)
print(score_L)

754.0
754.0


In [ ]:
em_score = sum([1 if gt == pred else 0 for gt, pred in zip(acts, preds)]) / len(acts) * 100
print(f"Exact Match (EM) Score: {em_score}%")


Exact Match (EM) Score: 0.1326259946949602%
